## In this note, we conisder oringal [[18,4,4]] code with 32 physical qubits, including 18 data qubits and 14 check qubits, as we remove two extra Z check qubits and two extra X check qubits.
## We simultaneously prepare all check qubits in the initial step and simultaneously measure all check qubits in the final step of a full cycle of the syndrome measurement circuit. A full cycle of the SM circuit contain 9 layers.
## We conisder no reset is perfomed after each syndrome measurement cycle.

In [1]:
import numpy as np
import random
from tabulate import tabulate
from scipy.sparse import coo_matrix, hstack 

from bposd.css import css_code
from ldpc import bposd_decoder, mod2
from mip import Model, xsum, minimize, BINARY, OptimizationStatus
import pickle

In [2]:
from scipy.optimize import fsolve
import itertools
from itertools import combinations
from itertools import chain
from functools import reduce

from scipy.io import savemat
from scipy.io import loadmat
from tabulate import tabulate

# from exp_BB  import equations, new_sum, p_ij, correlation_nodes
from exp_ldpc import rank2, distance_test, simulate_circuitZ, simulate_circuitX, generate_noisy_circuit
# from exp_BB import revised_BB_code, SM_circuit, decoding_matrix_BB_code, num_end, simulate_Zerror_syndrome, simulate_Xerror_syndrome
from exp_BB import *

Testing CSS code...
<Unnamed CSS code>, (3,6)-[[18,4,nan]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (3,6)-[[18,4,nan]]
Done
Computing code distance...
Code parameters: n,k,d= 18 4 4
circuit test: OK


## Data from simulation of noisy SM circuits¶ 
(we do not reset check qubits during the experiment)

In [60]:
# error_rate_init = 0.000
# error_rate_idle = 0.000
# error_rate_cnot = 0.003
# error_rate_meas = 0.000
# num_trials = 100000

In [26]:
# number of Monte Carlo trials
num_trials = 8000
max_cycles = 2; 

# depolarizing noise model 
error_rate_init = 0.000
error_rate_idle = 0.000
error_rate_cnot = 0.002
error_rate_meas = 0.002

## no reset for check qubits after each SM cycle
cycle_no_reset = cycle[2*n2-len(rx)-len(rz):]

In [27]:
Set_Zerror_syndrome_history = {} ;
Set_Zerror_syndrome_final_logical = {} ;

for num_cycles in range(1, max_cycles):

    # full syndrome measurement circuit
    # cycle_no_reset_repeated = num_cycles * cycle_no_reset

    Zerror_syndrome_history, Zerror_syndrome_final_logical =   \
        simulate_Zerror_syndrome(num_trials, num_cycles, cycle_no_reset, error_rate_init, error_rate_idle, error_rate_cnot, error_rate_meas) ;

    Set_Zerror_syndrome_history[num_cycles] = Zerror_syndrome_history
    Set_Zerror_syndrome_final_logical[num_cycles] = Zerror_syndrome_final_logical

In [28]:
Set_Xerror_syndrome_history = {} ;
Set_Xerror_syndrome_final_logical = {} ;

for num_cycles in range(1, max_cycles):

    # full syndrome measurement circuit
    # cycle_no_reset_repeated = num_cycles * cycle_no_reset

    Xerror_syndrome_history, Xerror_syndrome_final_logical =   \
        simulate_Xerror_syndrome(num_trials, num_cycles, cycle_no_reset, error_rate_init, error_rate_idle, error_rate_cnot, error_rate_meas) ;

    Set_Xerror_syndrome_history[num_cycles] = Xerror_syndrome_history
    Set_Xerror_syndrome_final_logical[num_cycles] = Xerror_syndrome_final_logical

## Calculate the error probability
based on the expectation values of detections in syndrome_history

In [29]:
# Set_Xerror_syndrome_list = {} ;
Set_Xerror_syndrome_probability_list = {}  ;
Set_Zerror_syndrome_probability_list = {}  ;

for num_cycles in range(1, max_cycles):
    decoding_matrix = decoding_matrix_BB_code(k, nbs, cycle, num_cycles, lin_order,data_qubits,Zchecks,Xchecks,
                                              error_rate_init,error_rate_idle,error_rate_cnot,error_rate_meas, lz, lx  )
    
    Xerror_syndrome_list = [ _ for _ in decoding_matrix.HXdict] ;
    Xerror_syndrome_list = [ tuple(num for num in item if num < (n2-len(rz))*(num_cycles + num_end) ) for item in Xerror_syndrome_list]
    Xerror_syndrome_history = Set_Xerror_syndrome_history[num_cycles] ;
    
    temp1 = DetectionToSyndromeProbability(num_trials, num_cycles, Xerror_syndrome_list, Xerror_syndrome_history)
    Set_Xerror_syndrome_probability_list[num_cycles] = temp1.syndrome_probability_list ;
#-----------------------------------------------------------------------------------------------------------------------------
    # Zerror_syndrome_list = [ _ for _ in decoding_matrix.HZdict] ;
    # Zerror_syndrome_list = [ tuple(num for num in item if num < (n2-len(rx))*(num_cycles + num_end) ) for item in Zerror_syndrome_list]
    # Zerror_syndrome_history = Set_Zerror_syndrome_history[num_cycles] ;

    # temp2 = DetectionToSyndromeProbability(num_trials, num_cycles, Zerror_syndrome_list, Zerror_syndrome_history)
    # Set_Zerror_syndrome_probability_list[num_cycles] = temp2.syndrome_probability_list ;

Number of Z noisy circuits= 344
Number of X noisy circuits= 344
Number of distinct X-syndrome histories= 89
Number of distinct Z-syndrome histories= 89


In [30]:
Set_Xerror_syndrome_probability_list
# list( Set_Xerror_syndrome_probability_list[1].values() )

{1: {(0, 7): 0.0051304828691203785,
  (1, 8): 0.004905680540628343,
  (2, 9): 0.0049866904046499405,
  (3, 10): 0.004845625461759817,
  (4, 11): 0.005225071813106941,
  (5, 12): 0.0051026334730490674,
  (6, 13): 0.006465391896400885,
  (0, 1, 4): 0.0017884788478847885,
  (1, 2, 5): 0.0018883888388838885,
  (0, 2, 6): 0.0004996399639963997,
  (0,): 0.0019501600174392443,
  (1, 3): 0.001430415847549667,
  (2, 3): 0.001765536572876341,
  (4, 5): 0.0019668466724757014,
  (5, 6): 0.0019810401541017256,
  (3, 4, 6): 0.0006095409540954096,
  (0, 2): 9.720447719068311e-05,
  (0, 1): 0.00012311065806265865,
  (1, 2, 3): 0.0006794779477947796,
  (3, 4): 0.0005762643247187599,
  (5,): 0.001893567594417913,
  (3, 6): 8.27081412574716e-05,
  (0, 4, 6): 0.0003198019801980198,
  (1, 4, 5): 0.0006395139513951396,
  (2, 5, 6): 0.0004097209720972097,
  (4, 6, 7): 0.001358865886588659,
  (4, 5, 8): 0.0019483348334833484,
  (5, 6, 9): 0.0015986498649864988,
  (1, 2, 10): 0.0017085508550855086,
  (3, 11): 

In [32]:
Set_Xerror_syndrome_probability_list[1]

{(0, 7): 0.0051304828691203785,
 (1, 8): 0.004905680540628343,
 (2, 9): 0.0049866904046499405,
 (3, 10): 0.004845625461759817,
 (4, 11): 0.005225071813106941,
 (5, 12): 0.0051026334730490674,
 (6, 13): 0.006465391896400885,
 (0, 1, 4): 0.0017884788478847885,
 (1, 2, 5): 0.0018883888388838885,
 (0, 2, 6): 0.0004996399639963997,
 (0,): 0.0019501600174392443,
 (1, 3): 0.001430415847549667,
 (2, 3): 0.001765536572876341,
 (4, 5): 0.0019668466724757014,
 (5, 6): 0.0019810401541017256,
 (3, 4, 6): 0.0006095409540954096,
 (0, 2): 9.720447719068311e-05,
 (0, 1): 0.00012311065806265865,
 (1, 2, 3): 0.0006794779477947796,
 (3, 4): 0.0005762643247187599,
 (5,): 0.001893567594417913,
 (3, 6): 8.27081412574716e-05,
 (0, 4, 6): 0.0003198019801980198,
 (1, 4, 5): 0.0006395139513951396,
 (2, 5, 6): 0.0004097209720972097,
 (4, 6, 7): 0.001358865886588659,
 (4, 5, 8): 0.0019483348334833484,
 (5, 6, 9): 0.0015986498649864988,
 (1, 2, 10): 0.0017085508550855086,
 (3, 11): 0.0016287616137969662,
 (12,): 0.

In [48]:
"""
X_detection_list   X_detection_way    X_detect_expectation    X_detection_probability
syndrome_list   syndrome_four_ele_property   detect_expectation    syndrome_probability_list
"""

'\nX_detection_list   X_detection_way    X_detect_expectation    X_detection_probability\nsyndrome_list   syndrome_four_ele_property   detect_expectation    syndrome_probability_list\n'

## Decoding error syndromes

In [56]:
Set_bpdX = {} ;
Set_bpdZ = {} ;
Set_HZ = {}  ;
Set_HX = {}  ;
for num_cycles in range(1, max_cycles):
    
    decoding_matrix = decoding_matrix_BB_code(k, nbs, cycle, num_cycles, lin_order, data_qubits, Zchecks, Xchecks,
                                        error_rate_init, error_rate_idle, error_rate_cnot, error_rate_meas, lz, lx)

    Set_HZ[num_cycles] = decoding_matrix.HZ ;  Set_HX[num_cycles] = decoding_matrix.HX ;
    
    bpdX = bposd_decoder(
        decoding_matrix.HdecX,  #the parity check matrix
        channel_probs = list(Set_Xerror_syndrome_probability_list[num_cycles].values()) ,
        max_iter=my_max_iter, bp_method=my_bp_method, ms_scaling_factor=my_ms_scaling_factor, osd_method=my_osd_method, 
        osd_order=my_osd_order 
        )
    Set_bpdX[num_cycles] = bpdX ;

    bpdZ = bposd_decoder(
        decoding_matrix.HdecZ,  
        channel_probs = list(Set_Zerror_syndrome_probability_list[num_cycles].values()) ,
        max_iter=my_max_iter, bp_method=my_bp_method, ms_scaling_factor=my_ms_scaling_factor, osd_method=my_osd_method,  
        osd_order=my_osd_order 
        )
    Set_bpdZ[num_cycles] = bpdZ ;

Number of Z noisy circuits= 344
Number of X noisy circuits= 344
Number of distinct X-syndrome histories= 89
Number of distinct Z-syndrome histories= 89


In [ ]:
# correct Z-type error
Z_error_correction = {} ;
Z_no_error_correction = {} ;
for num_cycles in range(1, max_cycles):

    Z_good_trials = 0
    Z_good_list = [0, 0, 0, 0] ;
    
    Z_no_correction_good_list = [0, 0, 0, 0] ;
    Z_no_correction_good_trials = 0
    
    for trial in range(num_trials):
        syndrome_history = Set_Zerror_syndrome_history[num_cycles][trial]
        syndrome_final_logical = Set_Zerror_syndrome_final_logical[num_cycles][trial]
        
        Z_no_correction_good_list += ( syndrome_final_logical == np.array([0, 0, 0, 0])  )
        if np.array_equal( syndrome_final_logical, np.array([0, 0, 0, 0]) ):
            assert(  all(cc == 0 for cc in syndrome_final_logical )  )
            Z_no_correction_good_trials+=1

        bpdZ = Set_bpdZ[num_cycles] ;
        bpdZ.decode(syndrome_history)
        low_weight_error = bpdZ.osdw_decoding
        
        syndrome_history_augmented_guessed = (Set_HZ[num_cycles] @ low_weight_error) % 2
        syndrome_final_logical_guessed = syndrome_history_augmented_guessed[len(syndrome_history):(len(syndrome_history)+k)]  
        
        ec_resultZ = np.array_equal(syndrome_final_logical_guessed, syndrome_final_logical)
        correct_z = (syndrome_final_logical_guessed + syndrome_final_logical)   % 2
        
        Z_good_list += ( correct_z == np.array([0, 0, 0, 0])  )
    
        if ec_resultZ:
            assert(  all(cc == 0 for cc in correct_z )  )
            Z_good_trials+=1
    
    print(Z_no_correction_good_trials/num_trials)
    print(Z_good_trials/num_trials)
    
    Z_error_correction[num_cycles] = {"logical error propability": Z_good_trials/num_trials,
                                      "good_trials": Z_good_trials,
                                      "good_list": Z_good_list } ;
    
    
    Z_no_error_correction[num_cycles] = {"error propability": Z_no_correction_good_trials/num_trials,
                                         "good_trials": Z_no_correction_good_trials,
                                         "good_list": Z_no_correction_good_list   } ;

In [57]:
# correct X-type error
X_error_correction = {} ;
X_no_error_correction = {} ;
for num_cycles in range(1, max_cycles):

    X_good_trials=0
    X_good_list = [0, 0, 0, 0] ;
    
    X_no_correction_good_list = [0, 0, 0, 0] ;
    X_no_correction_good_trials = 0
    
    for trial in range(num_trials):
        syndrome_history = Set_Xerror_syndrome_history[num_cycles][trial]
        syndrome_final_logical = Set_Xerror_syndrome_final_logical[num_cycles][trial]
        
        X_no_correction_good_list += ( syndrome_final_logical == np.array([0, 0, 0, 0])  )
        if np.array_equal( syndrome_final_logical, np.array([0, 0, 0, 0]) ):
            assert(  all(cc == 0 for cc in syndrome_final_logical )  )
            X_no_correction_good_trials+=1

        bpdX = Set_bpdX[num_cycles] ;
        bpdX.decode(syndrome_history)
        low_weight_error = bpdX.osdw_decoding
        
        syndrome_history_augmented_guessed = (Set_HX[num_cycles] @ low_weight_error) % 2
        syndrome_final_logical_guessed = syndrome_history_augmented_guessed[len(syndrome_history):(len(syndrome_history)+k)]  
        
        ec_resultX = np.array_equal(syndrome_final_logical_guessed, syndrome_final_logical)
        correct_x = (syndrome_final_logical_guessed + syndrome_final_logical)   % 2
        
        X_good_list += ( correct_x == np.array([0, 0, 0, 0])  )
    
        if ec_resultX:
            assert(  all(cc == 0 for cc in correct_x )  )
            X_good_trials+=1
    
    print(X_no_correction_good_trials/num_trials)
    print(X_good_trials/num_trials)
    
    X_error_correction[num_cycles] = {"logical error propability": X_good_trials/num_trials,
                                      "good_trials": X_good_trials,
                                      "good_list": X_good_list } ;
    
    
    X_no_error_correction[num_cycles] = {"error propability": X_no_correction_good_trials/num_trials,
                                         "good_trials": X_no_correction_good_trials,
                                         "good_list": X_no_correction_good_list   } ;    

0.89027
0.99575


## Save the data

In [ ]:
# savemat('Set_syndrome_history.mat', 
#         {  'Correct_X_trial_syndrome_history': Correct_X_trial_syndrome_history
#         })